## Evaluation of prediction results

 <a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/tutorials/07_Quality_assurance.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In the previous tutorial, we looked at various ways to visualise the results of our model.
These are useful for evaluating a linkage pipeline because they allow us to understand how our model works and verify that it is doing something sensible. They can also be useful to identify examples where the model is not performing as expected.

In addition to these spot checks, Splink also has functions to perform more formal accuracy analysis. These functions allow you to understand the likely prevalence of false positives and false negatives in your linkage models.

They rely on the existence of a sample of labelled (ground truth) matches, which may have been produced (for example) by human beings. For the accuracy analysis to be unbiased, the sample should be representative of the overall dataset.


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev

In [2]:
# Rerun our predictions to we're ready to view the charts
from splink import Linker, DuckDBAPI, splink_datasets

import pandas as pd

pd.options.display.max_columns = 1000

db_api = DuckDBAPI()
df = splink_datasets.fake_1000

In [3]:
import json
import urllib

url = "https://raw.githubusercontent.com/moj-analytical-services/splink_demos/master/demo_settings/saved_model_from_demo.json"

with urllib.request.urlopen(url) as u:
    settings = json.loads(u.read().decode())


linker = Linker(df, settings, database_api=DuckDBAPI())
df_predictions = linker.predict(threshold_match_probability=0.2)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'email':
    m values not fully trained


## Load in labels

The labels file contains a list of pairwise comparisons which represent matches and non-matches.

The required format of the labels file is described [here](https://moj-analytical-services.github.io/splink/linkerqa.html#splink.linker.Linker.roc_chart_from_labels).


In [4]:
from splink.datasets import splink_dataset_labels

df_labels = splink_dataset_labels.fake_1000_labels
df_labels.head(5)
labels_table = linker.register_labels_table(df_labels)

## Receiver operating characteristic curve

A [ROC chart](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) shows how the number of false positives and false negatives varies depending on the match threshold chosen. The match threshold is the match weight chosen as a cutoff for which pairwise comparisons to accept as matches.


In [5]:
linker.roc_chart_from_labels_table(labels_table)

alt.Chart(...)

### Precision-recall chart

An alternative representation of truth space is called a [precision recall curve](https://stats.stackexchange.com/questions/7207/roc-vs-precision-and-recall-curves).

This can be plotted as follows:


In [6]:
linker.precision_recall_chart_from_labels_table(labels_table)

alt.Chart(...)

## Truth table

Finally, Splink can also report the underlying table used to construct the ROC and precision recall curves.


In [7]:
roc_table = linker.truth_space_table_from_labels_table(labels_table)
roc_table.as_pandas_dataframe(limit=5)

,truth_threshold,match_probability,row_count,p,n,tp,tn,fp,fn,P_rate,N_rate,tp_rate,tn_rate,fp_rate,fn_rate,precision,recall,specificity,npv,accuracy,f1,f2,f0_5,p4,phi
0,-26.441763,1.097075e-08,3176.0,2031.0,1145.0,2031.0,0.0,1145.0,0.0,0.639484,0.360516,1.0,0.000000,1.000000,0.0,0.639484,1.0,0.000000,1.0,0.639484,0.780104,0.898673,0.689175,0.000000,0.000000
1,-25.336928,2.359526e-08,3176.0,2031.0,1145.0,2031.0,47.0,1098.0,0.0,0.639484,0.360516,1.0,0.041048,0.958952,0.0,0.649089,1.0,0.041048,1.0,0.654282,0.787209,0.902426,0.698082,0.143357,0.163229
2,-24.370660,4.609994e-08,3176.0,2031.0,1145.0,2031.0,154.0,991.0,0.0,0.639484,0.360516,1.0,0.134498,0.865502,0.0,0.672071,1.0,0.134498,1.0,0.687972,0.803879,0.911089,0.719244,0.366200,0.300653
3,-24.369413,4.613980e-08,3176.0,2031.0,1145.0,2031.0,199.0,946.0,0.0,0.639484,0.360516,1.0,0.173799,0.826201,0.0,0.682230,1.0,0.173799,1.0,0.702141,0.811102,0.914782,0.728531,0.433861,0.344341
4,-23.939180,6.217111e-08,3176.0,2031.0,1145.0,2031.0,230.0,915.0,0.0,0.639484,0.360516,1.0,0.200873,0.799127,0.0,0.689409,1.0,0.200873,1.0,0.711902,0.816154,0.917344,0.735071,0.474565,0.372134


!!! note "Further Reading"
:material-tools: For more on the quality assurance tools in Splink, please refer to the [Evaluation API documentation](../../linkereval.md).

    :bar_chart: For more on the charts used in this tutorial, please refer to the [Charts Gallery](../../charts/index.md#model-evaluation).

    :material-thumbs-up-down: For more on the Evaluation Metrics used in this tutorial, please refer to the [Edge Metrics guide.](../../topic_guides/evaluation/edge_metrics.md)


## :material-flag-checkered: That's it!

That wraps up the Splink tutorial! Don't worry, there are still plenty of resources to help on the next steps of your Splink journey:

:octicons-link-16: For some end-to-end notebooks of Splink pipelines, check out our [Examples](../examples/examples_index.md)

:simple-readme: For more deepdives into the different aspects of Splink, and record linkage more generally, check out our [Topic Guides](../../topic_guides/topic_guides_index.md)

:material-tools: For a reference on all the functionality avalable in Splink, see our [Documentation](../../documentation_index.md)
